In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df_final = pd.read_parquet("/Users/fahad/development/thesis/final_v2.parquet")

In [ ]:
df_final.head()

In [5]:
df_final.columns

Index(['id', 'cleanedContent', 'label', 'category', 'emotion_label', 'text',
       'text_without_stopwords', 'text_lemmatized',
       'text_without_stopwords_freq', 'date', 'rawContent',
       'probability.negative', 'probability.neutral', 'probability.positive',
       'probability.anger', 'probability.anticipation', 'probability.fear',
       'probability.joy', 'probability.optimism', 'probability.sadness',
       'Normalized_sentiment_score'],
      dtype='object')

In [6]:
df_final = df_final[['category', 'Normalized_sentiment_score']]

NameError: name 'df' is not defined

# Sentiment Correlation

In [ ]:
df_final = df_final[['probability.negative', 'probability.neutral', 'probability.positive']]

In [ ]:
corr = df_final.corr()

plt.figure(figsize=(20, 10))

sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
sns.clustermap(corr)

In [ ]:
df_final['sum'] = df_final.sum(axis=1)

In [ ]:
df_final['sum'] < 1

# Emotion Correlation

In [ ]:
df_final = df_final[['probability.anger', 'probability.anticipation', 'probability.disgust',
       'probability.fear', 'probability.joy', 'probability.love',
       'probability.optimism', 'probability.pessimism', 'probability.sadness',
       'probability.surprise', 'probability.trust']]

In [ ]:
corr = df_final.corr()

In [ ]:
plt.figure(figsize=(20, 10))

sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
sns.clustermap(corr)

optimism = optimism + trust ; remove trust
anticipation = anticipation + surprise ; remove surprise
anger = anger + disgust ; remove disgust
joy = joy + love ; remove love


In [ ]:
df_final.head()

In [ ]:
df_final['emotion_label'].value_counts()

In [ ]:
df_final['probability.joy'] = df_final['probability.joy'] + df_final['probability.love']
df_final['probability.anger'] = df_final['probability.anger'] + df_final['probability.disgust']
df_final['probability.anticipation'] = df_final['probability.anticipation'] + df_final['probability.surprise']
df_final['probability.optimism'] = df_final['probability.optimism'] + df_final['probability.trust']
df_final['probability.sadness'] = df_final['probability.sadness'] + df_final['probability.pessimism']

In [ ]:
df_final.drop(columns=['probability.disgust', 'probability.surprise', 'probability.trust', 'probability.love', 'probability.pessimism'], inplace=True)

In [ ]:
df_final.head()

In [ ]:
corr = df_final.corr()
plt.figure(figsize=(20, 10))

sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
df_final['emotion_label'].replace({'love': 'joy'}, inplace=True)

In [ ]:
df_final['emotion_label'].replace({'disgust': 'anger'}, inplace=True)

In [ ]:
df_final['emotion_label'].replace({'surprise': 'anticipation'}, inplace=True)

In [ ]:
df_final.to_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\final_v2.parquet")

# Normalize Sentiment

In [ ]:
df_sentiment = df_final[['id', 'probability.positive', 'probability.neutral', 'probability.negative', 'label']]

In [ ]:
# Define weights
W_positive = 1
W_neutral = 0.5
W_negative = 0

In [ ]:
# Calculate sentiment score
df_final['Sentiment_score'] = (W_positive * df_final['probability.positive'] + 
                         W_neutral * df_final['probability.neutral'] + 
                         W_negative * df_final['probability.negative'])


In [ ]:
# Normalize sentiment score to the range from -1 to 1
df_final['Normalized_sentiment_score'] = ((df_final['Sentiment_score'] - df_final['Sentiment_score'].min()) / 
                                     (df_final['Sentiment_score'].max() - df_final['Sentiment_score'].min())) * 2 - 1


In [ ]:
df_final.drop(columns='Sentiment_score', inplace=True)

In [ ]:
df_sentiment.to_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\normalized_sentiment.parquet")

# Normalize Emotions

In [ ]:
df_final = pd.read_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\final_v2.parquet")

In [ ]:
df_final.columns

In [ ]:
df_emotion = df_final[['probability.anger', 'probability.anticipation', 'probability.fear',
       'probability.joy', 'probability.optimism', 'probability.sadness', 'emotion_label']]

In [ ]:
corr = df_emotion.corr()
plt.figure(figsize=(20, 10))

sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
W_anger = -1
W_anticipation = 0
W_fear = -0.3
W_joy = 1
W_optimism = 0.7
W_sadness = -0.7

In [ ]:
W_anger = 0
W_anticipation = 0.5
W_fear = 0.1
W_joy = 1
W_optimism = 0.7
W_sadness = 0.2

In [ ]:
# Calculate sentiment score
df_final['emotion_score'] = (W_anger * df_final['probability.anger'] + 
                         W_anticipation * df_final['probability.anticipation'] + 
                         W_fear * df_final['probability.fear'] + 
                         W_joy * df_final['probability.joy'] + 
                         W_optimism * df_final['probability.optimism'] + 
                         W_sadness * df_final['probability.sadness'])

In [ ]:
# Normalize sentiment score to the range from -1 to 1
df_final['Normalized_emotion_score'] = ((df_final['emotion_score'] - df_final['emotion_score'].min()) / 
                                     (df_final['emotion_score'].max() - df_final['emotion_score'].min())) * 2 - 1

In [ ]:
df_final.to_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\final_v2.parquet")

# Ordinary Least Squares

In [2]:
df_final = pd.read_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\final_v2.parquet")

In [3]:
grouped = df_final.groupby('category').mean()

C:\Users\fashaikh\AppData\Local\Temp\ipykernel_10272\2236038258.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = df_final.groupby('category').mean()


In [ ]:
df_demo = pd.read_csv(r"C:\Users\fashaikh\Desktop\Thesis_main\thesis\acp_data\semi_cleaned_demo.csv")

In [ ]:
df_demo.head()

In [10]:
df_demo.columns

Index(['Community', 'Access to Exercise Opportunities',
       'Driving Alone to Work', 'Food Insecurity',
       'Limited Access to Healthy Foods', 'Long Commute - Driving Alone',
       '% 65 and Older', '% American Indian or Alaska Native', '% Asian',
       '% Below 18 Years of Age', '% Female', '% Hispanic',
       '% Native Hawaiian or Other Pacific Islander', '% Non-Hispanic Black',
       '% Non-Hispanic White', '% Not Proficient in English', '% Rural',
       'Bachelor's degree', 'Census Participation', 'High School Completion',
       'High School Graduation', 'School Segregation', 'Adult Obesity',
       'Adult Smoking', 'Alcohol-Impaired Driving Deaths',
       'Diabetes Prevalence', 'Excessive Drinking', 'Flu Vaccinations',
       'Frequent Mental Distress', 'Frequent Physical Distress',
       'Insufficient Sleep', 'Low Birthweight', 'Mammography Screening',
       'Physical Inactivity', 'Poor or Fair Health', 'Uninsured Adults',
       'Uninsured Children', 'Uninsured', 

In [ ]:
corr = grouped.corr()
plt.figure(figsize=(20, 10))
sns.heatmap(corr, annot=True)
plt.show()

In [6]:
grouped = grouped.merge(df_demo, 'inner', right_on= 'Community', left_on='category')

In [ ]:
sns.regplot(x='Insufficient Sleep', 
y='probability.joy', data=grouped)

In [ ]:
grouped.head()

In [7]:
y = grouped['Normalized_sentiment_score']

In [97]:
x = grouped[[
    # 'Community', 
    # 'Access to Exercise Opportunities',
    # 'Driving Alone to Work', 
    # 'Food Insecurity',
    # 'Limited Access to Healthy Foods', 
    # 'Long Commute - Driving Alone',
    # '% 65 and Older', 
    # '% American Indian or Alaska Native', 
    # '% Asian',
    # '% Below 18 Years of Age', 
    # '% Female', 
    # '% Hispanic',
    # '% Native Hawaiian or Other Pacific Islander', 
    # '% Non-Hispanic Black',
    # '% Non-Hispanic White', 
    # '% Not Proficient in English', 
    # '% Rural',
    # 'Bachelor\'s degree', 
    # 'Census Participation', 
    #  'High School Completion',
    # 'High School Graduation', 
    #  'School Segregation', 
    # 'Adult Obesity',
    # 'Adult Smoking', 
    # 'Alcohol-Impaired Driving Deaths',
    # 'Diabetes Prevalence', 
    # 'Excessive Drinking', 
    # 'Flu Vaccinations',
    # 'Frequent Mental Distress', 
    # 'Frequent Physical Distress',
     #'Insufficient Sleep', 
    # 'Low Birthweight', 
    # 'Mammography Screening',
    #'Physical Inactivity', 
    # 'Poor or Fair Health', 
    # 'Uninsured Adults',
    # 'Uninsured Children', 
    # 'Uninsured', 
    # 'Broadband Access', 
    # 'Democrat',
    # 'Voter Turnout', 
    # 'Child Care Cost Burden',
    # 'Children Eligible for Free or Reduced Price Lunch',
    # 'Children in Poverty', 
    # 'Children in Single-Parent Households',
    # 'Gender Pay Gap', 
    # 'Homeownership', 
    # 'Severe Housing Cost Burden',
    # 'Severe Housing Problems', 
     'Unemployment'
       ]]
x = sm.add_constant(x)

In [102]:
results = sm.OLS(y, x).fit()
# print(results.summary().as_text())
df = results.summary2().tables[1]

c:\Users\fashaikh\Desktop\Thesis_main\thesis\.venv\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [84]:
with open(r"C:\Users\fashaikh\Desktop\Thesis_main\thesis\statistical_modeling\statistically_significant\bachelors_census.txt", 'w') as fh:
        fh.write(results.summary().as_text())

c:\Users\fashaikh\Desktop\Thesis_main\thesis\.venv\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [ ]:
# Initialize the model
model = sm.OLS(y, x).fit()

In [ ]:
# Start backward elimination process
while True:
    max_p_value = model.pvalues.drop('const').max()  # Exclude p-value for the intercept

    if max_p_value > 0.05:
        max_var = model.pvalues.drop('const').idxmax()  # Variable with highest p-value
        x = X.drop(max_var, axis=1)  # Remove the variable with highest p-value
        model = sm.OLS(y, x).fit()  # Fit the updated model
    else:
        break

# Print the final summary
print(model.summary())


In [ ]:
Y = grouped['Normalized_sentiment_score']
X = grouped['Median Household Income']

In [ ]:
test_df = df_final

In [ ]:
test_df = test_df[['emotion_label', 'category']]

In [ ]:
dummies = pd.get_dummies(test_df, columns=['emotion_label'], drop_first=True)

In [ ]:
dummies

In [ ]:
test_df = dummies.merge(df_demo, 'left', left_on='category', right_on='Community')

In [ ]:
test_df.columns

In [ ]:
test_df = test_df[['category', 'emotion_label_anticipation',
       'emotion_label_fear', 'emotion_label_joy', 'emotion_label_optimism',
       'emotion_label_sadness', 'Unemployment']]

In [ ]:
y = test_df['Normalized_sentiment_score']
x = test_df[['Unemployment', 'Bachelor\'s degree']]
x = sm.add_constant(x)

In [ ]:
results = sm.OLS(y, x).fit()
print(results.summary())

# OLS Dummy